In [2]:
import os
import copy
import numpy as np
import pandas as pd
import random
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model

In [5]:
window = 200
slide = 0.1
train_test = 0.8
def make_windows_PCA(window, slide, frame, train_test):
    Viable_Cols = ['0', '1', '2', '3', '4', '5']
    Viable_Cols2 = ['0_max_freq', '1_max_freq', '2_max_freq', '3_max_freq', '4_max_freq', '5_max_freq', ]
    jump = window*slide
    train_samples = []
    sparse_train = []
    test_samples = []
    sparse_test = []
    labels_train = []
    labels_test = []
    if 'SPM/RPM' in frame.columns:
        label = 'SPM/RPM'
    if 'RPM/SPM' in frame.columns:
        label = 'RPM/SPM'
    if 'RPM' in frame.columns:
        label = 'RPM'
    if 'SPM' in frame.columns:
        label = 'SPM'
    for i in range(int(len(frame['0'])/(window*slide))-int((1/slide))):
        sample = frame.loc[(i*jump)+1:(i*jump)+(window), Viable_Cols]
        sample2 = []
        freqs = frame.loc[(i*jump)+200, Viable_Cols2]
        sample2.append(freqs.tolist())
        mins = [np.min(sample[col]) for col in Viable_Cols]
        maxes = [np.max(sample[col]) for col in Viable_Cols]
        stdev = [np.std(sample[col], axis=0) for col in Viable_Cols]
        means = [np.mean(sample[col]) for col in Viable_Cols]
        sample2.append(mins)
        sample2.append(maxes)
        sample2.append(stdev)
        sample2.append(means)
        yval = frame[label][(i*jump)+199]
        if i*jump <= (len(frame['0'])*train_test):
            train_samples.append(sample)
            sparse_train.append(sample2)
            labels_train.append(yval)
        else:
            test_samples.append(sample)
            sparse_test.append(sample2)
            labels_test.append(yval)
    labels_test = labels_test[9:]
    test_samples = test_samples[9:]
    sparse_test = sparse_test[9:]
    return train_samples, sparse_train, test_samples, sparse_test, labels_train, labels_test

def reshaper(frame):
    data_2d = frame.reshape(-1, frame.shape[-1])
    scaler = StandardScaler()
    whitened_data_2d = scaler.fit_transform(data_2d)
    whitened_frame = whitened_data_2d.reshape(frame.shape)
    return whitened_frame

In [ ]:
files = os.listdir('./PCA Fourier/')
bike_train_samples, bike_sparse_train, bike_test_samples, bike_sparse_test, bike_labels_train, bike_labels_test = [],[],[],[],[],[]
walk_train_samples, walk_sparse_train, walk_test_samples, walk_sparse_test, walk_labels_train, walk_labels_test = [],[],[],[],[],[]
run_train_samples, run_sparse_train, run_test_samples, run_sparse_test, run_labels_train, run_labels_test = [],[],[],[],[],[]

Biking_Files = [0,1,2,16,17]
for i in Biking_Files:
    print('A')
    frame = pd.read_csv('./PCA Fourier/'+str(files[i]))
    train_samples, sparse_train, test_samples, sparse_test, labels_train, labels_test = make_windows_PCA(window, slide, frame, train_test)
    bike_train_samples.extend(train_samples)
    bike_sparse_train.extend(sparse_train)
    bike_test_samples.extend(test_samples)
    bike_sparse_test.extend(sparse_test)
    bike_labels_train.extend(labels_train)
    bike_labels_test.extend(labels_test)
Walking_Files = [10,11,12,13,14,18,20,21]
for i in Walking_Files:
    print('B')
    frame = pd.read_csv('./PCA Fourier/'+str(files[i]))
    train_samples, sparse_train, test_samples, sparse_test, labels_train, labels_test = make_windows_PCA(window, slide, frame, train_test)
    walk_train_samples.extend(train_samples)
    walk_sparse_train.extend(sparse_train)
    walk_test_samples.extend(test_samples)
    walk_sparse_test.extend(sparse_test)
    walk_labels_train.extend(labels_train)
    walk_labels_test.extend(labels_test)
Running_Files = [4,5,6,7]
for i in Running_Files:
    print('C')
    frame = pd.read_csv('./PCA Fourier/'+str(files[i]))
    train_samples, sparse_train, test_samples, sparse_test, labels_train, labels_test = make_windows_PCA(window, slide, frame, train_test)
    run_train_samples.extend(train_samples)
    run_sparse_train.extend(sparse_train)
    run_test_samples.extend(test_samples)
    run_sparse_test.extend(sparse_test)
    run_labels_train.extend(labels_train)
    run_labels_test.extend(labels_test)

In [ ]:
tf_run_sparse_train = np.array(run_sparse_train)
tf_run_labels_train = np.array(run_labels_train)
tf_run_sparse_test = np.array(run_sparse_test)
tf_run_labels_test = np.array(run_labels_test)

whitened_run_sparse_train = reshaper(tf_run_sparse_train)
whitened_run_sparse_test = reshaper(tf_run_sparse_test)

def grid_search_run():
    epochs = [i+1 for i in range(10)]
    learning_rates = [0.1, 0.05, 0.03, 0.02, 0.01, 0.005, 0.002, 0.001]
    dropouts = [0.0, 0.1, 0.2, 0.5]
    histories = {}
    for epoch in epochs:
        for lr in learning_rates:
            for drop in dropouts:
                model_run = Sequential([
                    Flatten(input_shape=(5, 6)),
                    Dense(64, activation='relu'),
                    Dropout(drop),
                    Dense(128, activation='relu'),
                    Dropout(drop),
                    Dense(1, activation='relu')
                ])
                model_run.compile(optimizer=Adam(learning_rate=lr), 
                               loss='mean_absolute_error', 
                               metrics=['mean_absolute_error'])
                history = model_run.fit(whitened_run_sparse_train, tf_run_labels_train, epochs=epoch)
                performance = model_run.evaluate(whitened_run_sparse_test, tf_run_labels_test)

                histories[f'Training Accuracy {epoch}, {lr}, {drop}'] = {
                    'Epochs': epoch,
                    'LR' : lr,
                    'Dropout' : drop,
                    'Training Error': history.history['mean_absolute_error'],
                    'Test Error': performance[1]
                }
    return histories

results = grid_search_run()
epochs = [i+1 for i in range(10)]
learning_rates = [0.1, 0.05, 0.03, 0.02, 0.01, 0.005, 0.002, 0.001]
dropouts = [0.0, 0.1, 0.2, 0.5]
dropout = [[],[],[], []]
learning_params = [[],[],[],[],[],[],[],[]]
epoch = [[],[],[],[],[],[],[],[],[],[]]
for key, value in results.items():
    epoch_key = value['Epochs']
    epoch[epochs.index(epoch_key)].append(value['Test Error'])
    drop_key = value['Dropout']
    dropout[dropouts.index(drop_key)].append(value['Test Error'])
    lr_key = value['LR']
    learning_params[learning_rates.index(lr_key)].append(value['Test Error'])


In [ ]:
tf_walk_sparse_train = np.array(walk_sparse_train)
tf_walk_labels_train = np.array(walk_labels_train)
tf_walk_sparse_test = np.array(walk_sparse_test)
tf_walk_labels_test = np.array(walk_labels_test)

whitened_walk_sparse_train = reshaper(tf_walk_sparse_train)
whitened_walk_sparse_test = reshaper(tf_walk_sparse_test)

def grid_search_walk():
    epochs = [i+1 for i in range(10)]
    learning_rates = [0.1, 0.05, 0.03, 0.02, 0.01, 0.005, 0.002, 0.001]
    dropouts = [0.0, 0.1, 0.2, 0.5]
    histories = {}
    for epoch in epochs:
        for lr in learning_rates:
            for drop in dropouts:
                model_walk = Sequential([
                    Flatten(input_shape=(5, 6)),
                    Dense(64, activation='relu'),
                    Dropout(drop),
                    Dense(128, activation='relu'),
                    Dropout(drop),
                    Dense(1, activation='relu')
                ])
                model_walk.compile(optimizer=Adam(learning_rate=lr), 
                               loss='mean_absolute_error', 
                               metrics=['mean_absolute_error'])
                history = model_walk.fit(whitened_walk_sparse_train, tf_walk_labels_train, epochs=epoch)
                performance = model_walk.evaluate(whitened_walk_sparse_test, tf_walk_labels_test)

                histories[f'Training Accuracy {epoch}, {lr}, {drop}'] = {
                    'Epochs': epoch,
                    'LR' : lr,
                    'Dropout' : drop,
                    'Training Error': history.history['mean_absolute_error'],
                    'Test Error': performance[1]
                }
    return histories

results2 = grid_search_walk()
dropout2 = [[],[],[],[]]
learning_params2 = [[],[],[],[],[],[],[],[]]
epoch2 = [[],[],[],[],[],[],[],[],[],[]]
for key, value in results2.items():
    epoch_key = value['Epochs']
    epoch2[epochs.index(epoch_key)].append(value['Test Error'])
    drop_key = value['Dropout']
    dropout2[dropouts.index(drop_key)].append(value['Test Error'])
    lr_key = value['LR']
    learning_params2[learning_rates.index(lr_key)].append(value['Test Error'])

In [ ]:
tf_bike_sparse_train = np.array(bike_sparse_train)
tf_bike_labels_train = np.array(bike_labels_train)
tf_bike_sparse_test = np.array(bike_sparse_test)
tf_bike_labels_test = np.array(bike_labels_test)

whitened_bike_sparse_train = reshaper(tf_bike_sparse_train)
whitened_bike_sparse_test = reshaper(tf_bike_sparse_test)

def grid_search_bike():
    epochs = [i+1 for i in range(10)]
    learning_rates = [0.1, 0.05, 0.03, 0.02, 0.01, 0.005, 0.002, 0.001]
    dropouts = [0.0, 0.1, 0.2, 0.5]
    histories = {}
    for epoch in epochs:
        for lr in learning_rates:
            for drop in dropouts:
                model_bike = Sequential([
                    Flatten(input_shape=(5, 6)),
                    Dense(64, activation='relu'),
                    Dropout(drop),
                    Dense(128, activation='relu'),
                    Dropout(drop),
                    Dense(1, activation='relu')
                ])
                model_bike.compile(optimizer=Adam(learning_rate=lr), 
                               loss='mean_absolute_error', 
                               metrics=['mean_absolute_error'])
                history = model_bike.fit(whitened_bike_sparse_train, tf_bike_labels_train, epochs=epoch)
                performance = model_bike.evaluate(whitened_bike_sparse_test, tf_bike_labels_test)

                histories[f'Training Accuracy {epoch}, {lr}, {drop}'] = {
                    'Epochs': epoch,
                    'LR' : lr,
                    'Dropout' : drop,
                    'Training Error': history.history['mean_absolute_error'],
                    'Test Error': performance[1]
                }
    return histories

results3 = grid_search_bike()
dropout3 = [[],[],[],[]]
learning_params3 = [[],[],[],[],[],[],[],[]]
epoch3 = [[],[],[],[],[],[],[],[],[],[]]
for key, value in results3.items():
    epoch_key = value['Epochs']
    epoch3[epochs.index(epoch_key)].append(value['Test Error'])
    drop_key = value['Dropout']
    dropout3[dropouts.index(drop_key)].append(value['Test Error'])
    lr_key = value['LR']
    learning_params3[learning_rates.index(lr_key)].append(value['Test Error'])

In [15]:
print("Run")
for k in range(4):
    print('Dropout '+str(dropouts[k])+' Max: '+str(min(dropout[k])))
for i in range(10):
    print('Epochs '+str(i+1)+' Max: '+str(min(epoch[i])))
for j in range(8):
    print('LR '+str(learning_rates[j])+' Max: '+str(min(learning_params[j])))

print("walk")
for k in range(4):
    print('Dropout '+str(dropouts[k])+' Max: '+str(min(dropout2[k])))
for i in range(10):
    print('Epochs '+str(i+1)+' Max: '+str(min(epoch2[i])))
for j in range(8):
    print('LR '+str(learning_rates[j])+' Max: '+str(min(learning_params2[j])))

print("Bike")
for k in range(4):
    print('Dropout '+str(dropouts[k])+' Max: '+str(min(dropout3[k])))
for i in range(10):
    print('Epochs '+str(i+1)+' Max: '+str(min(epoch3[i])))
for j in range(8):
    print('LR '+str(learning_rates[j])+' Max: '+str(min(learning_params3[j])))

Run
Dropout 0.0 Max: 5.514613628387451
Dropout 0.1 Max: 4.438214302062988
Dropout 0.2 Max: 6.748065948486328
Dropout 0.5 Max: 5.853658199310303
Epochs 1 Max: 12.555818557739258
Epochs 2 Max: 12.137567520141602
Epochs 3 Max: 7.866607666015625
Epochs 4 Max: 6.923299312591553
Epochs 5 Max: 6.332035541534424
Epochs 6 Max: 6.070057392120361
Epochs 7 Max: 8.256141662597656
Epochs 8 Max: 4.438214302062988
Epochs 9 Max: 5.9169182777404785
Epochs 10 Max: 5.2158522605896
LR 0.1 Max: 4.438214302062988
LR 0.05 Max: 5.514613628387451
LR 0.03 Max: 5.853658199310303
LR 0.02 Max: 5.9169182777404785
LR 0.01 Max: 5.817226409912109
LR 0.005 Max: 7.185391902923584
LR 0.002 Max: 9.97131061553955
LR 0.001 Max: 12.70941162109375
walk
Dropout 0.0 Max: 6.18964147567749
Dropout 0.1 Max: 7.189887523651123
Dropout 0.2 Max: 7.430885314941406
Dropout 0.5 Max: 8.448270797729492
Epochs 1 Max: 9.501120567321777
Epochs 2 Max: 8.568853378295898
Epochs 3 Max: 7.920655727386475
Epochs 4 Max: 7.615570545196533
Epochs 5 Max